In [1]:
# Import libraries
import pandas as pd

### Scrape 2020 Player Salaries

In [2]:
# Loop through all pages for URL and scrape tables

# Set range for pages
pages = []

for i in range(2, 19):
    pages.append(i)

# Scrape tables for 2020 page 1
url_2020 = 'http://www.espn.com/nba/salaries/_/seasontype/3'
tables_2020 = pd.read_html(url_2020)[0]
tables_2020['year'] = '2020'

# # Set variable for dataframe
tables_2020_pgs_df = pd.DataFrame()

# # Set URL variable for all other pages
for page in pages:
    url_2020_pgs = f'http://www.espn.com/nba/salaries/_/page/{page}/seasontype/3'
    tables_2020_pgs = pd.read_html(url_2020_pgs)[0]
    tables_2020_pgs['year'] = '2020'
    tables_2020_pgs_df = pd.concat(axis = 0, objs=[tables_2020_pgs, tables_2020_pgs_df])
    

In [3]:
len(tables_2020)

44

In [4]:
len(tables_2020_pgs_df)

541

### Scrape all other years

In [5]:
years = []

for i in range(2000, 2020):
    years.append(i)
years

[2000,
 2001,
 2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019]

In [6]:
# tables_all_others = []
tables_all_others_df = pd.DataFrame()

# Scrape tables for all other years - page 1
for year in years:
    url_all_others = f'http://www.espn.com/nba/salaries/_/year/{year}/seasontype/3'
    tables_all_others = pd.read_html(url_all_others)[0]
    tables_all_others['year'] = year
    tables_all_others_df = pd.concat(axis = 0, objs=[tables_all_others, tables_all_others_df])



In [8]:
len(tables_all_others_df)

880

In [9]:
years = []

for i in range(2000, 2020):
    years.append(i)

tables_all_others_pgs_df = pd.DataFrame()

# Scrape tables for all other years - all other pages
for year in years:
    for page in pages:
        url_all_others_pgs = f'http://www.espn.com/nba/salaries/_/year/{year}/page/{page}/seasontype/3'
        tables_all_others_pgs = pd.read_html(url_all_others_pgs)[0]
        tables_all_others_pgs['year'] = year
        tables_all_others_pgs_df = pd.concat(axis = 0, objs=[tables_all_others_pgs, tables_all_others_pgs_df])


In [10]:
len(tables_all_others_pgs_df)

9077

In [11]:
# Concat all dataframes
all_salaries_df = pd.concat(axis = 0, objs=[tables_2020, tables_2020_pgs_df, tables_all_others_df, tables_all_others_pgs_df])
len(all_salaries_df)

10542

In [14]:
# Clean dataframe

# Add column headers
all_salaries_df.columns = ['Rank', 'Player_Position', 'Team', 'Salary', 'Year']

# Drop na
all_salaries_df = all_salaries_df.dropna()

# Drop extra rows
drop_row = all_salaries_df[ (all_salaries_df['Rank'] == 'RK') & (all_salaries_df['Player_Position'] == 'NAME') & (all_salaries_df['Team'] == 'TEAM') & (all_salaries_df['Salary'] == 'SALARY')].index
all_salaries_df.drop(drop_row, inplace=True)

# Split Player_Position into 2 columns
all_salaries_df['Player'], all_salaries_df['Position'] = all_salaries_df['Player_Position'].str.split(',', 1).str



# Convert salary to interger
all_salaries_df['Salary'] = all_salaries_df['Salary'].replace('[\$,]','',regex=True).astype(int)
all_salaries_df['Rank'] = all_salaries_df['Rank'].astype(int)
all_salaries_df['Year'] = all_salaries_df['Year'].astype(int)

C:\Users\pompa\anaconda3\envs\usc_bootcamp\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  


In [18]:
len(all_salaries_df)

9456

In [19]:
# Drop Player_Position column
all_salaries_df = all_salaries_df.drop('Player_Position', axis = 1)


KeyError: "['Player_Position'] not found in axis"

In [20]:
all_salaries_df = all_salaries_df.sort_values(['Year', 'Rank'], ascending = (False, True))
                                              

## Adjust salary for inflation

In [21]:
# !pip install cpi

In [22]:
import cpi


C:\Users\pompa\anaconda3\envs\usc_bootcamp\lib\site-packages\cpi\__init__.py:46: StaleDataWarning: CPI data is out of date. To accurately inflate to today's dollars, you must run `cpi.update()`.
  warnings.warn(StaleDataWarning())


In [23]:
adjusted_salaries_df = all_salaries_df.loc[(all_salaries_df['Year'] != 2020)&(all_salaries_df['Year'] != 2019)]


In [24]:
adjusted_salaries_df['Inflation'] = adjusted_salaries_df.apply(lambda x: cpi.inflate(x.Salary, x.Year), axis=1)


C:\Users\pompa\anaconda3\envs\usc_bootcamp\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [25]:
adjusted_salaries_df['Inflation'] = adjusted_salaries_df['Inflation'].astype(int)
adjusted_salaries_df.tail()

C:\Users\pompa\anaconda3\envs\usc_bootcamp\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Rank,Team,Salary,Year,Player,Position,Inflation
36,153,Indiana Pacers,745000,2000,Al Harrington,PF,1086380
37,154,Miami Heat,728000,2000,Tim James,F,1061590
38,155,Philadelphia 76ers,684000,2000,Jumaine Jones,SF,997428
39,156,Utah Jazz,679000,2000,Scott Padgett,PF,990137
40,157,Milwaukee Bucks,301000,2000,Rafer Alston,PG,438926


In [26]:
current_salaries_df = all_salaries_df.loc[(all_salaries_df['Year'] == 2020)|(all_salaries_df['Year'] == 2019)]
current_salaries_df['Inflation'] = current_salaries_df['Salary']


C:\Users\pompa\anaconda3\envs\usc_bootcamp\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
final_salaries_df = pd.concat([current_salaries_df, adjusted_salaries_df], axis=0).reset_index(drop=True)

In [31]:
final_salaries_df

,Rank,Team,Salary,Year,Player,Position,Inflation
0,1,Golden State Warriors,40231758,2020,Stephen Curry,PG,40231758
1,2,Oklahoma City Thunder,38506482,2020,Chris Paul,PG,38506482
2,3,Houston Rockets,38506482,2020,Russell Westbrook,PG,38506482
3,4,Washington Wizards,38199000,2020,John Wall,PG,38199000
4,5,Brooklyn Nets,38199000,2020,Kevin Durant,SF,38199000
...,...,...,...,...,...,...,...
9451,153,Indiana Pacers,745000,2000,Al Harrington,PF,1086380
9452,154,Miami Heat,728000,2000,Tim James,F,1061590
9453,155,Philadelphia 76ers,684000,2000,Jumaine Jones,SF,997428
9454,156,Utah Jazz,679000,2000,Scott Padgett,PF,990137


In [34]:
final_salaries_df = final_salaries_df[['Year', 'Rank', 'Player', 'Position', 'Salary', 'Inflation']]
final_salaries_df

,Year,Rank,Player,Position,Salary,Inflation
0,2020,1,Stephen Curry,PG,40231758,40231758
1,2020,2,Chris Paul,PG,38506482,38506482
2,2020,3,Russell Westbrook,PG,38506482,38506482
3,2020,4,John Wall,PG,38199000,38199000
4,2020,5,Kevin Durant,SF,38199000,38199000
...,...,...,...,...,...,...
9451,2000,153,Al Harrington,PF,745000,1086380
9452,2000,154,Tim James,F,728000,1061590
9453,2000,155,Jumaine Jones,SF,684000,997428
9454,2000,156,Scott Padgett,PF,679000,990137


## Save to csv

In [35]:
final_salaries_df.to_csv('nba_salaries.csv', index=False)

In [36]:
final_salaries_df

,Year,Rank,Player,Position,Salary,Inflation
0,2020,1,Stephen Curry,PG,40231758,40231758
1,2020,2,Chris Paul,PG,38506482,38506482
2,2020,3,Russell Westbrook,PG,38506482,38506482
3,2020,4,John Wall,PG,38199000,38199000
4,2020,5,Kevin Durant,SF,38199000,38199000
...,...,...,...,...,...,...
9451,2000,153,Al Harrington,PF,745000,1086380
9452,2000,154,Tim James,F,728000,1061590
9453,2000,155,Jumaine Jones,SF,684000,997428
9454,2000,156,Scott Padgett,PF,679000,990137
